In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.dates as dates

import os 
import random
from contextlib import contextmanager
from time import time
from tqdm import tqdm

import plotly.express as px 
import plotly.graph_objects as go 
from plotly.subplots import make_subplots

import lightgbm as lgbm
import category_encoders as ce 
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

ModuleNotFoundError: No module named 'plotly'